<a href="https://colab.research.google.com/github/Sairam954/CASES2022/blob/main/ScalabilityAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Below Scalability Analysis Code is developed using Equation 8 and Equation 11 from "***Scaling up silicon photonic-based accelerators: Challenges and opportunities***"

Steps:
1. Use Equation 8 to find the Photodetector sensitivity for a bit precision and datarate. **findPDSensitivity** method performs this step, it sweeps the PD sensitivity and finds the bit precision computed by equation 8. The method returns the PD Sensistivy for which the difference between the computed and required precision is minimum.  
2. Use Equation 11 to find the maximum N and M supported for given bitprecision. **findOptimalN** method performs this step, it sweeps N and M values and calculates the output power available at PD, the N and M values that have output power POut greater than PD sensitivity calculated in step 1 are supported.

3. Save the results in a csv file.





PD Sensitivity

In [3]:
import numpy as np
def findPDSensitivity(no_of_bits, DR):
    e = 1.6*10**(-19)  # columbs
    KT = 0.0259*e  # columbs*Volt
    R = 1.2  # A/W
    Id = 35*10**(-9)  # A
    RL = 50  # ohm
    DR = DR*10**(9)  # Bits/s
    RIN = 10**(-140/10)  # power ratio/ Hz

    Pd_range = np.arange(-35, 0, 0.1)
    error_list = []
    pd_list = []
    for pd_dbm in Pd_range:
        Pd = 10**((pd_dbm-30)/10)  # W
        A = R*Pd
        B = 2*e*(R*Pd+Id)
        C = (4*KT)/RL
        D = (R**2)*(Pd**2)*RIN
        E = 2*e*Id + C
        F = DR/np.sqrt(2)
        no_of_bits_hat = (
            1/6.02)*(20*np.log10(A/((np.sqrt(B+C+D)+np.sqrt(E))*np.sqrt(F)))-1.76)
        error = abs(no_of_bits_hat - no_of_bits)
        error_list.append(error)
        pd_list.append(pd_dbm)

    min_error = min(error_list)
    min_error_idx = error_list.index(min_error)
    pd_dbm = pd_list[min_error_idx]
    print("*******Calculated PD Sensitivity*****", pd_dbm)
    return pd_dbm

In [7]:
import numpy as np
def findOptimalN(PLaser, pd_dbm, arch = "MAW"):
    # # Optical Power Calculation
    Psmf_att = 0
    Pec_il = 1.6
    Psi_att = 0.3
    Pmrm_ip_il = 4
    Pmrm_ip_obl = 0.01
    Psplitter_il = 0.01
    Pmrr_w_il = 0.01
    Pmrr_w_obl = 0.01
    dMRR = 0.02;
    if arch == 'MAW':
      p_penalty = 4.8 #
    elif arch == 'AMW':
      p_penalty = 5.8
    elif arch == 'MMA':
      p_penalty = 1.8
    n_range = 201
    N = range(1,n_range)
    Pout =  0
    maxN =1
    for n in N:
        m = n
        Pout = PLaser - Psmf_att - Pec_il - (Psi_att*n*dMRR) - Pmrm_ip_il - (n-1)*Pmrm_ip_obl - (10*np.log10(n)+Psplitter_il*np.log2(m)) -Pmrr_w_il -(n-1)*Pmrr_w_obl - p_penalty

        if Pout<pd_dbm:
          break
        else:
          maxN = n
    # for M not equal to N
    # m_range = 300
    # M = range(1,m_range)
    # Pout =  0
    # maxM =1
    # for m in M:
    #     n = 4
    #     Pout = PLaser - Psmf_att - Pec_il - (Psi_att*n*dMRR) - Pmrm_ip_il - (n-1)*Pmrm_ip_obl - (10*np.log10(n)+Psplitter_il*np.log2(m)) -Pmrr_w_il -(n-1)*Pmrr_w_obl - p_penalty
    #     if Pout<pd_dbm:
    #       break
    #     else:
    #       maxM = m
    print("*******Calculated Max Supported N*****", maxN )
    return maxN, Pout

Below cell performs steps 1, 2, and 3. Please update the parameters according to your requirement.

**NOTE**: In step 2, ***findOptimalN*** assumes M=N, you have to update the code if N is not equal to M.

In [8]:
import pandas as pd

bits_range = [1,2,3,4,5,6,7,8] # UPDATE the list of bit precision you wish to extract N and M
DR_range = [1, 3, 5, 10 ] # UPDATE the list of datarate
PLaser = 10 # dBm # Input Laser Power
arch = 'MMA' # Three type of architectures AMW 'DEAPCNN', MAW 'HOLYLIGHT', and MMA
result_list = []
for no_of_bits in bits_range:
    for DR in DR_range:
        result = {}
        Pd_dbm = findPDSensitivity(no_of_bits, DR)
        N, Pout = findOptimalN(PLaser, Pd_dbm, arch)
        # result['PD_Sensitivity'] = Pd
        result['N'] = N
        # result['PLaser'] = PLaser
        result['Pout'] = Pout
        result['no_of_bits'] = no_of_bits
        result['DR'] = DR
        result_list.append(result)
df = pd.DataFrame(result_list)
df.to_csv('MMA_N_Recieved_Power.csv') # UPDATE the file name

*******Calculated PD Sensitivity***** -26.999999999999886
*******Calculated Max Supported N***** 200
*******Calculated PD Sensitivity***** -24.699999999999854
*******Calculated Max Supported N***** 180
*******Calculated PD Sensitivity***** -23.499999999999837
*******Calculated Max Supported N***** 156
*******Calculated PD Sensitivity***** -21.999999999999815
*******Calculated Max Supported N***** 130
*******Calculated PD Sensitivity***** -23.999999999999844
*******Calculated Max Supported N***** 166
*******Calculated PD Sensitivity***** -21.59999999999981
*******Calculated Max Supported N***** 123
*******Calculated PD Sensitivity***** -20.499999999999794
*******Calculated Max Supported N***** 106
*******Calculated PD Sensitivity***** -18.999999999999773
*******Calculated Max Supported N***** 85
*******Calculated PD Sensitivity***** -20.9999999999998
*******Calculated Max Supported N***** 114
*******Calculated PD Sensitivity***** -18.599999999999767
*******Calculated Max Supported N****